In [2]:
import os
from DSSCircuit_Interface import * 
import networkx as nx

In [3]:
FolderName=os.path.dirname(os.path.realpath("__file__"))
d_obj= DSS(r""+ FolderName+ "\ieee34Mod2.dss")  #creating a DSS object instance

In [4]:
d_obj.compile_ckt_dss() #compiling the circuit
# d_obj.solve_snapshot_dss(1.0) #solving snapshot power flow
d_obj.dssSolution.Solve()

In [5]:
edges_dictlist=[] #dictionary for edge information
G_original=nx.Graph() #undirected graph
for e in d_obj.get_AllPDElements():
    if ((e.split('.')[0]=='Line') or (e.split('.')[0]=='Transformer')): #only if its a line or transformer used in graph(capacitors avoided)
        branch_obj=Branch(d_obj,e) #creating  a branch object instance with full name
        sr_node=branch_obj.bus_fr.split('.')[0] #extracting source bus of branch
        tar_node=branch_obj.bus_to.split('.')[0] #extracting target bus of branch
        name=e.split('.')[1] #getting the name of the element(label or id)
        el_type=e.split('.')[0] #getting the type of the element (line or transformer)
        numphases=branch_obj.nphases #extracting number of phases of branch
        Inorm=branch_obj.Cap #extracting the base network (normal operation at max load) current flow in branch
        Imax=branch_obj.MaxCap #extracting the maximum permissible current flow in branch
        Iresid=Imax-Inorm #residual current flow in branch
        if el_type== 'Line':
            R,X=d_obj.get_Line_RX(name)
        else:
            R,X=d_obj.get_Transformer_RX(name)
        edges_dictlist.append({"source":sr_node, "target":tar_node, "label":name, "device":el_type, "resistance":R, "reactance":X, "phases":numphases,"maxcap":Imax,"cap":Inorm,"residue":Iresid})
        G_original.add_edge(sr_node, tar_node, label=name, device=el_type, resistance=R, reactance=X, phases=numphases,maxcap=Imax,cap=Inorm,residue=Iresid)

In [6]:
buses=list(d_obj.get_AllBuses())
loads=d_obj.get_AllLoads()
for b in buses:
    P_sum=0
    Q_sum=0
    for j in range(len(loads)):
        if loads[j]["Bus"]==b:
            P_sum=P_sum+loads[j].get("Pload")
            Q_sum=Q_sum+loads[j].get("Qload")
    G_original.nodes[b]["Device"]="Bus"
    G_original.nodes[b]["TotConnected_ActiveLoad"]=P_sum
    G_original.nodes[b]["TotConnected_ReactiveLoad"]=Q_sum

In [7]:
G_original.edges(data=True)

EdgeDataView([('sourcebus', '800', {'label': 'subxf', 'device': 'Transformer', 'resistance': 0.012400199999999998, 'reactance': 0.024800399999999997, 'phases': 3, 'maxcap': 313.7773202117531, 'cap': 16.50637143694331, 'residue': 297.27094877480977}), ('800', '802', {'label': 'l1', 'device': 'Line', 'resistance': 0.5473215904400001, 'reactance': 0.40716439247999997, 'phases': 3, 'maxcap': 600.0, 'cap': 45.77337931904375, 'residue': 554.2266206809562}), ('802', 'mid806', {'label': 'l2a', 'device': 'Line', 'resistance': 0.1835012309033333, 'reactance': 0.13651054244, 'phases': 3, 'maxcap': 600.0, 'cap': 45.779600967482736, 'residue': 554.2203990325172}), ('mid806', '806', {'label': 'l2b', 'device': 'Line', 'resistance': 0.1835012309033333, 'reactance': 0.13651054244, 'phases': 3, 'maxcap': 600.0, 'cap': 44.51783080175414, 'residue': 555.4821691982459}), ('806', '808', {'label': 'l3', 'device': 'Line', 'resistance': 6.837277077473332, 'reactance': 5.086398592879998, 'phases': 3, 'maxcap': 

In [8]:
G_original.nodes(data=True)

NodeDataView({'sourcebus': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '800': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '802': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, 'mid806': {'Device': 'Bus', 'TotConnected_ActiveLoad': 55.0, 'TotConnected_ReactiveLoad': 29.0}, '806': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '808': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, 'mid810': {'Device': 'Bus', 'TotConnected_ActiveLoad': 16.0, 'TotConnected_ReactiveLoad': 8.0}, '810': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '812': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '814': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad': 0}, '814r': {'Device': 'Bus', 'TotConnected_ActiveLoad': 0, 'TotConnected_ReactiveLoad'

In [9]:
nx.readwrite.gml.write_gml(G_original,"34busEx.gml") #Graph undirected with edge features and node features which are constant